🧪 Colab Notebook — QLoRA Fine-Tuning on Mistral-7B
Below is a full Colab-compatible notebook script to:

Load your reviews CSV

Group and summarize product data

Create an instruction-style dataset

Fine-tune mistralai/Mistral-7B-v0.1 using QLoRA and SFTTrainer

In [2]:
# Install Required Libraries
#!pip install openai

#### Import Librairies

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI
import pandas as pd
import time
from IPython.display import display
from tqdm.notebook import tqdm
from collections import defaultdict
import json
from datetime import datetime

## Setup

In [4]:
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Debug check:
print("✅ API Key Loaded:", bool(OPENAI_API_KEY))

✅ API Key Loaded: True


In [5]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)


#### Load and Format Your Dataset

In [6]:
# Load your dataset
df = pd.read_csv("data/cleaned_amazon_reviews_final.csv")

# Optional: limit size for test
#df = df.sample(10)  # or remove this line to use all

df.columns


Index(['name', 'asins', 'brand', 'categories', 'primaryCategories',
       'imageURLs', 'doRecommend', 'rating', 'sourceURLs', 'label',
       'clean_text', 'name_cleaned', 'umap_kmeans_cluster', 'cluster_labels',
       'cluster_confidence', 'topic', 'topic_prob', 'cluster_id', 'tag',
       'is_best_deal', 'is_family_friendly', 'is_durability_issue',
       'is_top_product', 'is_battery_praise', 'is_battery_complaint',
       'is_performance_met', 'final_tag'],
      dtype='object')

In [7]:
df.shape

(20060, 27)

In [8]:
df.columns

Index(['name', 'asins', 'brand', 'categories', 'primaryCategories',
       'imageURLs', 'doRecommend', 'rating', 'sourceURLs', 'label',
       'clean_text', 'name_cleaned', 'umap_kmeans_cluster', 'cluster_labels',
       'cluster_confidence', 'topic', 'topic_prob', 'cluster_id', 'tag',
       'is_best_deal', 'is_family_friendly', 'is_durability_issue',
       'is_top_product', 'is_battery_praise', 'is_battery_complaint',
       'is_performance_met', 'final_tag'],
      dtype='object')

#### Build products_by_cluster dictionary (used for prompt constraints)

In [9]:
# Group all product names by cluster_labels or primaryCategories
products_by_cluster = (
    df.groupby("primaryCategories")["name"]
    .unique()
    .apply(list)
    .to_dict()
)


#### Prompts function - JSON Instruction with Product Constraints

In [10]:
def build_prompt(category, product_stats, product_list, min_reviews=3):
    """
    Generates a JSON-format instruction prompt for GPT based on review statistics of a product category.
    
    Args:
        category (str): The product category or cluster (e.g. 'Fire Tablet', 'Kindle Accessories').
        product_stats (DataFrame): A DataFrame grouped by product name, with:
            - 'rating': average rating for each product
            - 'clean_text': concatenated or sampled review text for each product
        min_reviews (int): Number of review snippets to include per product (default: 3).
    
    Returns:
        str: A fully formatted GPT prompt with:
            - List of product names to constrain hallucination
            - Review snippets and average ratings
            - Instruction to return clean, valid JSON (no markdown, no prefix)
    """

    # Step 1: Get top and worst-rated products
    top_3 = product_stats.head(3)
    worst = product_stats.tail(1)

    # Step 2: Build allowed product name list (used to constrain product listing)
    product_list = "\n".join([f"- {name}" for name in product_list])

    # Step 3: Start assembling the instruction prompt
    prompt_text = f"""
You are an expert Amazon product analyst.

You are evaluating products in the "{category}" category. Here is a list of **real product names** that you are allowed to mention (do not make up products):
{product_list}

---

Your task is to return a **valid, minified, single-line JSON** using the following format:

{{
  "top_products": [
    {{
      "name": "<Product name>",
      "rating": <float>,
      "highlight": "<Why this product is recommended>",
      "tag": "<tag>"
    }}
  ],
  "worst_product": {{
    "name": "<Product name>",
    "rating": <float>,
    "issue": "<Why it should be avoided>",
    "tag": "<tag>"
  }},
  "summary_text": "<Optional 1-2 sentence wrap-up of the category>"
}}

Guidelines:
- If there are fewer than 3 top products, list available ones and explain in `summary_text`.
- If there's no obvious worst product, return `"worst_product": null`.
- You MUST include all keys exactly as shown.
- Do NOT break or omit any closing brackets or quotes.

Format constraints:
- Return only valid JSON.
- Do not include markdown, explanations, or formatting.
- JSON **must be a single line**, compact, and fully closed.
- Example output: `{{"top_products":[{{"name":"X","rating":4.5,...}}],...}}`


---

Here are summarized reviews and average ratings:
"""

    # Step 4: Insert top product review data
    for i, (name, row) in enumerate(top_3.iterrows(), 1):
        prompt_text += f"\n{i}. {name}\n"
        prompt_text += f"   - Rating: {round(row['rating'], 1)}\n"
        prompt_text += f"   - Sample Reviews: {row['clean_text'][:350]}...\n"
        prompt_text += f"   - Tag: {row['final_tag']}\n"

    # Step 5: Insert worst product summary
    for name, row in worst.iterrows():
        prompt_text += f"\nWorst Product: {name}\n"
        prompt_text += f"   - Rating: {round(row['rating'], 1)}\n"
        prompt_text += f"   - Sample Reviews: {row['clean_text'][:350]}...\n"
        prompt_text += f"   - Tag: {row['final_tag']}\n"

    return prompt_text.strip()


#### Generate Prompts per Category

In [11]:
prompt_data = []

for category, group in df.groupby("primaryCategories"):  # or "primaryCategories"
    product_stats = (
        group.groupby("name")
        .agg({
            "rating": "mean",
            "clean_text": lambda x: " ".join(x.sample(min(3, len(x))).tolist()),
            "final_tag": lambda x: x.value_counts().index[0]  # most frequent tag
        })
        .sort_values(by="rating", ascending=False)
    )

    if len(product_stats) < 1:
        continue

    product_list = products_by_cluster[category]
    prompt_text = build_prompt(category, product_stats, product_list)
    prompt_data.append({"category": category, "prompt": prompt_text})


#### Convert prompts to DataFrame

In [12]:
df_prompts = pd.DataFrame(prompt_data)

#### Display a preview

In [13]:
display(df_prompts.head())

,category,prompt
0,Animals & Pet Supplies,You are an expert Amazon product analyst.\n\nY...
1,Electronics,You are an expert Amazon product analyst.\n\nY...
2,"Electronics,Furniture",You are an expert Amazon product analyst.\n\nY...
3,"Electronics,Hardware",You are an expert Amazon product analyst.\n\nY...
4,"Electronics,Media",You are an expert Amazon product analyst.\n\nY...


#### OpenAI API Query Function

In [14]:
def query_gpt(prompts, model="gpt-3.5-turbo", temperature=0.7, max_tokens=300, sleep=1.0):
    if isinstance(prompts, str):
        prompts = [prompts]

    responses = []
    for prompt in tqdm(prompts, desc="Querying GPT"):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=temperature,
                max_tokens=max_tokens
            )
            content = response.choices[0].message.content.strip()
        except Exception as e:
            content = f"[ERROR] {str(e)}"
        responses.append(content)
        time.sleep(sleep)

    return responses if len(responses) > 1 else responses[0]

#### Validate JSON Responses with Fallback

In [15]:
def safe_json_parse(text):
    try:
        return json.loads(text)
    except Exception:
        return None


In [16]:
import json

def parse_gpt_output_safe(text):
    if not isinstance(text, str):
        return None
    try:
        # Remove tuple wrapping if needed
        if text.startswith("('") and text.endswith("')"):
            text = text[2:-2]

        # Trim whitespace
        text = text.strip()

        # Force close with '"})' if missing
        if not text.endswith('"}\')'):
            text = text.rstrip()[:-4] + '"}\')'

        # Now extract the JSON inside the string wrapper
        if text.startswith("{") and text.endswith("}"):
            return json.loads(text)
        elif text.startswith("('") and text.endswith("')"):
            return json.loads(text[2:-2])
        elif text.startswith("{") or text.startswith('"top_products"'):
            return json.loads(text)

    except Exception as e:
        print("❌ Parse failed:", e)
        print("Raw string:\n", text[:300], "...")
        return None




#### Send to GPT

In [17]:
# Query GPT and get outputs
df_prompts["gpt_output"] = query_gpt(df_prompts["prompt"].tolist())

Querying GPT:   0%|          | 0/10 [00:00<?, ?it/s]

In [18]:
# Try to parse the GPT response into JSON
df_prompts["gpt_json"] = df_prompts["gpt_output"].apply(safe_json_parse)

# Save results to CSV
#df_prompts.to_csv("gpt_category_summaries.csv", index=False)

# Preview a few outputs
display(df_prompts[["category", "gpt_output","gpt_json"]].head())


,category,gpt_output,gpt_json
0,Animals & Pet Supplies,"{""top_products"":[{""name"":""AmazonBasics Single-...",None
1,Electronics,"{""top_products"":[{""name"":""AmazonBasics USB 3.0...",{'top_products': [{'name': 'AmazonBasics USB 3...
2,"Electronics,Furniture","{""top_products"":[{""name"":""All-new Echo (2nd Ge...",{'top_products': [{'name': 'All-new Echo (2nd ...
3,"Electronics,Hardware","{""top_products"":[{""name"":""Amazon - Echo Plus w...",{'top_products': [{'name': 'Amazon - Echo Plus...
4,"Electronics,Media","{""top_products"":[{""name"":""Kindle Oasis E-reade...",None


In [19]:
import pprint

# Example: check the 0th row (first category)
pprint.pprint(df_prompts.loc[4, "gpt_json"])

None


#### Enrich GPT outputs with image URLs

In [20]:
def enrich_gpt_json_with_images(gpt_dict, df_reference):
    """
    Adds imageURL to each product in top_products and worst_product by matching product names in df_reference.
    
    Args:
        gpt_dict (dict): A single GPT output dict.
        df_reference (DataFrame): Reference DataFrame containing 'name' and 'imageURLs'.
    
    Returns:
        dict: The enriched dictionary.
    """
    def find_image_url(product_name):
        # Try exact match first
        match = df_reference[df_reference["name"].str.strip() == product_name.strip()]
        if not match.empty:
            return match.iloc[0]["imageURLs"]
        
        # Fallback: partial match
        match = df_reference[df_reference["name"].str.contains(product_name[:30], case=False, na=False)]
        if not match.empty:
            return match.iloc[0]["imageURLs"]
        
        return None

    # Enrich top_products
    for product in gpt_dict.get("top_products", []):
        if "name" in product:
            product["imageURL"] = find_image_url(product["name"])

    # Enrich worst_product
    worst = gpt_dict.get("worst_product")
    if isinstance(worst, dict) and "name" in worst:
        worst["imageURL"] = find_image_url(worst["name"])

    return gpt_dict


In [21]:
df_prompts["gpt_json"] = df_prompts["gpt_json"].apply(
    lambda g: enrich_gpt_json_with_images(g, df) if g is not None else None
)


In [22]:
df_prompts

,category,prompt,gpt_output,gpt_json
0,Animals & Pet Supplies,You are an expert Amazon product analyst.\n\nY...,"{""top_products"":[{""name"":""AmazonBasics Single-...",None
1,Electronics,You are an expert Amazon product analyst.\n\nY...,"{""top_products"":[{""name"":""AmazonBasics USB 3.0...",{'top_products': [{'name': 'AmazonBasics USB 3...
2,"Electronics,Furniture",You are an expert Amazon product analyst.\n\nY...,"{""top_products"":[{""name"":""All-new Echo (2nd Ge...",{'top_products': [{'name': 'All-new Echo (2nd ...
3,"Electronics,Hardware",You are an expert Amazon product analyst.\n\nY...,"{""top_products"":[{""name"":""Amazon - Echo Plus w...",{'top_products': [{'name': 'Amazon - Echo Plus...
4,"Electronics,Media",You are an expert Amazon product analyst.\n\nY...,"{""top_products"":[{""name"":""Kindle Oasis E-reade...",None
5,Health & Beauty,You are an expert Amazon product analyst.\n\nY...,"{""top_products"":[{""name"":""AmazonBasics AAA Per...",{'top_products': [{'name': 'AmazonBasics AAA P...
6,Home & Garden,You are an expert Amazon product analyst.\n\nY...,"{""top_products"":[{""name"":""AmazonBasics Nespres...",{'top_products': [{'name': 'AmazonBasics Nespr...
7,Office Supplies,You are an expert Amazon product analyst.\n\nY...,"{""top_products"":[{""name"":""Expanding Accordion ...",{'top_products': [{'name': 'Expanding Accordio...
8,"Office Supplies,Electronics",You are an expert Amazon product analyst.\n\nY...,"{""top_products"":[{""name"":""Fire HD 10 Tablet, 1...","{'top_products': [{'name': 'Fire HD 10 Tablet,..."
9,"Toys & Games,Electronics",You are an expert Amazon product analyst.\n\nY...,"{""top_products"":[{""name"":""Fire Kids Edition Ta...",{'top_products': [{'name': 'Fire Kids Edition ...


In [23]:
pprint.pprint(df_prompts.loc[4,"gpt_json"])

None


#### Save the JSON

In [24]:
# Make sure df_prompts["gpt_json"] contains valid dicts
summaries = df_prompts["gpt_json"].tolist()

# Save to JSON file
with open("gpt_summaries_by_category.json", "w", encoding="utf-8") as f:
    json.dump(summaries, f, indent=2, ensure_ascii=False)